# Imports

In [1]:
import time
import numpy as np
import pandas as pd

# Constant

In [12]:
DATASET_PATH = '/kaggle/input/zocdoc/DAC_NationalDownloadableFile.csv'

# Dictionary of doctors
dict_drs = {
    "MD": "Doctor of Medicine",
    "DO": "Doctor of Osteopathic Medicine",
    "DC": "Doctor of Chiropractic",
    "OD": "Doctor of Optometry",
    "DPM": "Doctor of Podiatric Medicine",
    "DDS": "Doctor of Dental Surgery",
    "DDM": "Doctor of Dental Medicine",
    "PSY": "Psychologist"
}

# Dictionary of nurses
dict_nurse = {
    "NP": "Nurse Practitioner",
    "CNA": "Certified Nursing Assistant",
    "CNS": "Clinical Nurse Specialist",
    "CNM": "Certified Nurse Midwife"
}

# Dictionary of assistants
dict_assistant = {
    "PA": "Physician Assistant",
    "AA": "Anesthesiologist Assistant"
}

# Code

### Step1: Load original national dataset

In [3]:
%%time
df = pd.read_csv(DATASET_PATH, low_memory=False)

CPU times: user 22.8 s, sys: 4.44 s, total: 27.3 s
Wall time: 31.8 s


In [4]:
df.head(4)

,NPI,Ind_PAC_ID,Ind_enrl_ID,Provider Last Name,Provider First Name,Provider Middle Name,suff,gndr,Cred,Med_sch,...,adr_ln_1,adr_ln_2,ln_2_sprs,City/Town,State,ZIP Code,Telephone Number,ind_assgn,grp_assgn,adrs_id
0,1790868909,42100042,I20100809000798,HAGHIGHI,BIJAN,NaN,NaN,M,NaN,UNIVERSITY OF SOUTHERN CALIFORNIA KECK SCHOOL ...,...,1 HOAG DR,NaN,NaN,NEWPORT BEACH,CA,926634162,9.498911e+09,Y,Y,CA926634162NE1XXXXDRXX300
1,1114033073,42100109,I20040319001256,JONES,TAMMY,JO,NaN,F,CNA,OTHER,...,327 MEDICAL PARK DR,NaN,NaN,BRIDGEPORT,WV,263309006,6.813423e+09,Y,Y,WV263309006BR327XXDRXX400
2,1508823618,42100117,I20050426000780,GRIFFIN,DAVID,NaN,NaN,M,MD,VANDERBILT UNIVERSITY SCHOOL OF MEDICINE,...,1 SAINT FRANCIS DR,NaN,NaN,GREENVILLE,SC,296013955,5.138539e+09,Y,Y,SC296013955GR1XXXXDRXX400
3,1508823618,42100117,I20050426000780,GRIFFIN,DAVID,NaN,NaN,M,MD,VANDERBILT UNIVERSITY SCHOOL OF MEDICINE,...,317 SAINT FRANCIS DR,SUITE 220,NaN,GREENVILLE,SC,296013976,8.642552e+09,Y,Y,SC296013976GR317XXDRXX402


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2572154 entries, 0 to 2572153
Data columns (total 31 columns):
 #   Column                Dtype  
---  ------                -----  
 0   NPI                   int64  
 1   Ind_PAC_ID            int64  
 2   Ind_enrl_ID           object 
 3   Provider Last Name    object 
 4   Provider First Name   object 
 5   Provider Middle Name  object 
 6   suff                  object 
 7   gndr                  object 
 8   Cred                  object 
 9   Med_sch               object 
 10  Grd_yr                float64
 11  pri_spec              object 
 12  sec_spec_1            object 
 13  sec_spec_2            object 
 14  sec_spec_3            object 
 15  sec_spec_4            object 
 16  sec_spec_all          object 
 17  Telehlth              object 
 18  Facility Name         object 
 19  org_pac_id            float64
 20  num_org_mem           float64
 21  adr_ln_1              object 
 22  adr_ln_2              object 
 23  ln_2_sp

In [11]:
df['Cred'].value_counts()

Cred
MD     401558
PA      38702
DO      24746
NP      24595
CNA     17890
DC      12660
OD      11942
PT      11915
DPM      8175
CSW      7153
CP       5816
OT       1284
AU       1123
CNS       883
MNT       778
CNM       774
AA        435
DDS       363
DDM       178
PSY       112
SCW         2
Name: count, dtype: int64

### Step2: Make new dataset using the national dataset attributes

In [22]:
%%time
new_df = pd.DataFrame({
    'enrl_id': df['Ind_enrl_ID'], # Extra
    'name': df['Provider First Name'] + ' ' + df['Provider Middle Name'].fillna('') + ' ' + df['Provider Last Name'],
    'job_title': df['pri_spec'],
    'rating': np.nan,
    'facility_name': df['Facility Name'],
    'street_address': df['adr_ln_1'].fillna('') + ' ' + df['adr_ln_2'].fillna(''),
    'address_locality': df['City/Town'],
    'region': df['State'],
    'postal_code': df['ZIP Code'],
    'is_dr': 0,
    'is_nurse': 0,
    'is_assistant': 0,
    'credential': df['Cred'],
})

CPU times: user 6.53 s, sys: 1.1 s, total: 7.63 s
Wall time: 7.64 s


In [19]:
new_df.head(3)

,enrl_id,name,job_title,rating,facility_name,street_address,address_locality,region,postal_code,is_dr,is_nurse,is_assistant,credential
0,I20100809000798,BIJAN HAGHIGHI,PATHOLOGY,NaN,NEWPORT HARBOR PATHOLOGY MEDICAL GROUP INC,1 HOAG DR,NEWPORT BEACH,CA,926634162,NaN,NaN,NaN,NaN
1,I20040319001256,TAMMY JO JONES,CERTIFIED REGISTERED NURSE ANESTHETIST (CRNA),NaN,UNITED HOSPITAL CENTER INC,327 MEDICAL PARK DR,BRIDGEPORT,WV,263309006,NaN,NaN,NaN,CNA
2,I20050426000780,DAVID GRIFFIN,GYNECOLOGICAL ONCOLOGY,NaN,BON SECOURS MEDICAL GROUP GREENVILLE SPECIALTY...,1 SAINT FRANCIS DR,GREENVILLE,SC,296013955,NaN,NaN,NaN,MD


### Step3: Impute values of 'is_dr', 'is_nurse', 'is_assistant'

In [23]:
%%time
for index, row in new_df.iterrows():
    credential = row['credential']

    if credential in dict_drs:
        new_df.at[index, 'is_dr'] = 1
    if credential in dict_nurse:
        new_df.at[index, 'is_nurse'] = 1
    if credential in dict_assistant:
        new_df.at[index, 'is_assistant'] = 1

CPU times: user 3min 7s, sys: 357 ms, total: 3min 8s
Wall time: 3min 8s


In [24]:
new_df.head(3)

,enrl_id,name,job_title,rating,facility_name,street_address,address_locality,region,postal_code,is_dr,is_nurse,is_assistant,credential
0,I20100809000798,BIJAN HAGHIGHI,PATHOLOGY,NaN,NEWPORT HARBOR PATHOLOGY MEDICAL GROUP INC,1 HOAG DR,NEWPORT BEACH,CA,926634162,0,0,0,NaN
1,I20040319001256,TAMMY JO JONES,CERTIFIED REGISTERED NURSE ANESTHETIST (CRNA),NaN,UNITED HOSPITAL CENTER INC,327 MEDICAL PARK DR,BRIDGEPORT,WV,263309006,0,1,0,CNA
2,I20050426000780,DAVID GRIFFIN,GYNECOLOGICAL ONCOLOGY,NaN,BON SECOURS MEDICAL GROUP GREENVILLE SPECIALTY...,1 SAINT FRANCIS DR,GREENVILLE,SC,296013955,1,0,0,MD


In [26]:
new_df = new_df.drop(columns=['credential'])

In [32]:
new_df = new_df[new_df['region'].isin(['NY', 'NJ', 'PA', 'DE'])]

In [33]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 379976 entries, 36 to 2572134
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   enrl_id           379976 non-null  object 
 1   name              379951 non-null  object 
 2   job_title         379976 non-null  object 
 3   rating            0 non-null       float64
 4   facility_name     350946 non-null  object 
 5   street_address    379976 non-null  object 
 6   address_locality  379976 non-null  object 
 7   region            379976 non-null  object 
 8   postal_code       379976 non-null  object 
 9   is_dr             379976 non-null  int64  
 10  is_nurse          379976 non-null  int64  
 11  is_assistant      379976 non-null  int64  
dtypes: float64(1), int64(3), object(8)
memory usage: 37.7+ MB


In [35]:
new_df.to_csv('df_dac_finalised.csv', index=False)